In [ ]:
%load_ext autoreload
%autoreload 2

In [70]:
from environs import Env
env = Env()
env.read_env('.env-upswell-vlab', override=True)

from adopt.malaria import get_conf, get_df, get_ad_token
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState

cnf = get_conf(env)
token = get_ad_token(cnf["survey_user"], cnf["chatbase"])

state = CampaignState(env, token)
m = Marketing(env, state)

# TODO: make new strata format conf parse
# TODO: make get_df work with new strata format


In [ ]:
state.adsets

In [71]:
from adopt.malaria import load_typed_json
from adopt.marketing import Audience
from typing import List

audiences = load_typed_json("config/audiences.json", List[Audience])

In [83]:
import pandas as pd


def load_cities(path):
    cities = pd.read_csv(path)
    cities = cities[cities.rad >= 1.0]
    return cities


df = pd.read_csv('output/cities.csv')

In [84]:
df = df[['disthash', 'rad', 'lng', 'lat', 'creative_group', 'include_audience']]

In [85]:
from facebook_business.adobjects.targetinggeolocationcustomlocation \
    import TargetingGeoLocationCustomLocation
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targeting import Targeting

def create_location(lat, lng, rad):
    return {
        TargetingGeoLocationCustomLocation.Field.latitude: lat,
        TargetingGeoLocationCustomLocation.Field.longitude: lng,
        TargetingGeoLocationCustomLocation.Field.radius: rad,
        TargetingGeoLocationCustomLocation.Field.distance_unit: 'kilometer',
    }

# include entry survey????

def make_stratum(facebook_targeting, quota, df):
    locs = [create_location(r.lat, r.lng, r.rad) 
           for _, r in df.iterrows()]

    targeting = {
        **facebook_targeting,
        Targeting.Field.geo_locations: {
            TargetingGeoLocation.Field.location_types: ['home'],
            TargetingGeoLocation.Field.custom_locations: locs
        },
        Targeting.Field.custom_audiences: [{"id": "23845214311790518"}],
        Targeting.Field.excluded_custom_audiences: [{"id": "23845214312520518"}],
    }

    id_ = df.disthash.iloc[0]

    return { 'id': id_,
             'metadata': {'stratumid': id_},
             'facebook_targeting': targeting, 
             'creatives': ['vlab-mnm-hindi-hand-black-white', 'vlab-mnm-hindi-grains'],
             'quota': quota, 
             'shortcodes': ['baselineng'],
             'target_questions': [
                 {'ref': 'md:stratumid', 'op': 'equal', 'field': 'response', 'value': id_},
                 {
                     'name': 'occupation',
                     'ref': '4fc929c7-132d-49b1-a164-515e5cc9064f',
                     'op': 'answered',
                     'field': 'response',
                     'value': None
                 },
             ]}


strata = df.groupby('disthash').apply(lambda df: make_stratum({'age_min': 20}, 100, df)).tolist()
    

In [86]:
import json
with open('config/test-strata.json', 'w') as f:
    f.write(json.dumps(strata))

In [87]:
from adopt.marketing import load_strata_conf

strata = load_strata_conf('config/test-strata.json')

In [88]:
from adopt.marketing import CreativeConf

creatives = load_typed_json("config/creatives-new.json", List[CreativeConf])

In [89]:
creatives

[CreativeConf(name='vlab-mnm-hindi-hand-black-white', image_hash='2ab26a332d793602881a1df770bddd05', image='hand-black-white.png', body='Hum yeh jaankar jeette hain ki aap kaise swasth rehte hain, aap 300 INR ke mobile credit prapt karake jeette hain!', welcome_message='Hum yeh jaankar jeette hain ki aap kaise swasth rehte hain, aap 300 INR ke mobile credit prapt karake jeette hain!', link_text='Messenger main survey shuroo karen!', button_text='Start!', form='langchoice'),
 CreativeConf(name='vlab-mnm-hindi-grains', image_hash='faef6b3c706ac05bcd6e23182c4059ee', image='grains.png', body='Hum yeh jaankar jeette hain ki aap kaise swasth rehte hain, aap 300 INR ke mobile credit prapt karake jeette hain!', welcome_message='Hum yeh jaankar jeette hain ki aap kaise swasth rehte hain, aap 300 INR ke mobile credit prapt karake jeette hain!', link_text='Messenger main survey shuroo karen!', button_text='Start!', form='langchoice')]

In [90]:
from adopt.malaria import hydrate_strata

strata = hydrate_strata(strata, creatives)

In [ ]:
df = get_df(cnf, strata)

In [ ]:
df

In [24]:
from adopt.clustering import _base_budget

budget = _base_budget(strata, 2000000, 10000)

In [25]:
instructions = [m.adset_instructions(s, budget[s.id]) for s in strata]

In [26]:
instructions = [x for y in instructions for x in y]

In [ ]:
instructions[:3]

In [ ]:
from adopt.facebook.update import GraphUpdater
import logging

updater = GraphUpdater(state)
    
for i in instructions:
    report = updater.execute(i)
    logging.info(report)

In [ ]:
strata[0]

In [ ]:
# Audiences

In [30]:
from adopt.audiences import create_audience
from adopt.marketing import create_custom_audience


i = create_custom_audience('vlab-test-custom-aud', "virtual lab auto-generated audience")

updater.execute(i)

{'timestamp': '2020-12-08T15:04:48.828872+00:00',
 'instruction': {'node': 'custom_audience',
  'action': 'create',
  'id': None,
  'params': {'name': 'vlab-test-custom-aud',
   'subtype': 'CUSTOM',
   'description': 'virtual lab auto-generated audience',
   'customer_file_source': 'USER_PROVIDED_ONLY'}}}

In [59]:
ca = state.custom_audiences[0]

In [ ]:
from facebook_business.adobjects.customaudience import CustomAudience

dir(CustomAudience.Field)

In [ ]:
state.custom_audiences

In [ ]:
ca

In [ ]:
# approximate_count > 1000


# just try to make lookalike aud until it succeeds...

# or make from UI? 

In [49]:
from adopt.audiences import create_lookalike

# n = lookalike_name(audiences[0])
i = create_lookalike(audiences[0], ca)

updater.execute(i)

KeyboardInterrupt: 